# 데이터 전처리

In [1]:
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk import pos_tag
from nltk.stem import PorterStemmer
import collections
from tensorflow.keras.datasets import imdb
import pickle

# 전처리
def MyPreprocessing(text):
    text2 = "".join([" " if ch in string.punctuation else ch for ch in text])
    tokens = nltk.word_tokenize(text2)
    tokens = [word.lower() for word in tokens]
    
    stopwds = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopwds]
    
    tokens = [word for word in tokens if len(word)>=3]
    
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    tagged_corpus = pos_tag(tokens)    
    
    Noun_tags = ['NN','NNP','NNPS','NNS']
    Verb_tags = ['VB','VBD','VBG','VBN','VBP','VBZ']

    lemmatizer = WordNetLemmatizer()

    def prat_lemmatize(token,tag):
        if tag in Noun_tags:
            return lemmatizer.lemmatize(token,'n')
        elif tag in Verb_tags:
            return lemmatizer.lemmatize(token,'v')
        else:
            return lemmatizer.lemmatize(token,'n')

    pre_proc_text =  " ".join([prat_lemmatize(token,tag) for token,tag in tagged_corpus])             

    return pre_proc_text

# IMDB 데이터에 사용된 총 단어의 종류는 88,584개 (vocabulary 크기)이다.
# 가장 많이 사용되는 6,000개 단어만 사용하고, 나머지는 OOV로 표시한다.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=6000,
                                                      start_char=0,
                                                      oov_char=0,
                                                      index_from=0)

# train, test 데이터를 합친다. 필요한 경우 나중에 나눠쓴다.
text = np.hstack([x_train, x_test])
label = np.hstack([y_train, y_test])

# vocabulary를 가져온다.
word2idx = imdb.get_word_index()
idx2word = dict((v,k) for k,v in word2idx.items())

# start_char와 oov_char는 '.'으로 표시해 둔다. 나중에 전처리 과정에서 제거된다.
idx2word[0] = '.'

# 숫자로 표시된 x_train을 실제 단어로 변환한다.
def decode(review):
    x = [idx2word[s] for s in review]
    return ' '.join(x)

# 리뷰 문서를 전처리한다.
reviews = []
for i, review in enumerate(text):
    reviews.append(MyPreprocessing(decode(review)))
    if i % 100 == 0: print(i)

# 전처리된 리뷰 문서로 vocabulary를 다시 생성한다.
counter = collections.Counter()
for line in reviews:
    for word in nltk.word_tokenize(line):
        counter[word.lower()] += 1

word2idx2 = {w:(i+1) for i, (w, _) in enumerate(counter.most_common())}
idx2word2 = {v:k for k,v in word2idx2.items()}

# # 전처리된 리뷰 문서와 vocabulary를 저장한다.
# with open('data/preprocessed_review.pickle', 'wb') as f:
#     pickle.dump([reviews, label, word2idx2, idx2word2], f, pickle.HIGHEST_PROTOCOL)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [2]:
# SkNS 학습. pre-trained embedding vector를 생성한다.
# -------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding
from tensorflow.keras.layers import Dot, Activation, Reshape
from tensorflow.keras.models import Model
import pickle
import nltk

# # 전처리가 완료된 IMDB 데이터를 읽어온다.
# with open('data/preprocessed_review.pickle', 'rb') as f:
#     reviews, label, word2idx, idx2word = pickle.load(f)

# Trigram으로 학습 데이터를 생성한다.
xs = []     # 입력 데이터
ys = []     # 출력 데이터
for line in reviews:   
    # 사전에 부여된 번호로 단어들을 표시한다.
    embedding = [word2idx[w.lower()] for w in nltk.word_tokenize(line)]
    
    # Trigram으로 주변 단어들을 묶는다.
    triples = list(nltk.trigrams(embedding))
    
    # 왼쪽 단어, 중간 단어, 오른쪽 단어로 분리한다.
    w_lefts = [x[0] for x in triples]
    w_centers = [x[1] for x in triples]
    w_rights = [x[2] for x in triples]
    
    # 입력 (xs)      출력 (xy)
    # ---------    -----------
    # 중간 단어 --> 왼쪽 단어
    # 중간 단어 --> 오른쪽 단어
    xs.extend(w_centers)
    ys.extend(w_lefts)
    xs.extend(w_centers)
    ys.extend(w_rights)

# SGNS용 학습 데이터를 생성한다.
rand_word = np.random.randint(1, len(word2idx), len(xs))
x_pos = np.vstack([xs, ys]).T
x_neg = np.vstack([xs, rand_word]).T

y_pos = np.ones(x_pos.shape[0]).reshape(-1,1)
y_neg = np.zeros(x_neg.shape[0]).reshape(-1,1)

x_total = np.vstack([x_pos, x_neg])
y_total = np.vstack([y_pos, y_neg])

X = np.hstack([x_total, y_total])
np.random.shuffle(X)

# SGNS 모델을 생성한다.
vocab_size = len(word2idx) + 1  # 사전의 크기
EMB_SIZE = 64
BATCH_SIZE = 10240
NUM_EPOCHS = 10

inputX = Input(batch_shape = (None, 1))
inputY = Input(batch_shape = (None, 1))
embX = Embedding(vocab_size, EMB_SIZE)(inputX)
embY = Embedding(vocab_size, EMB_SIZE)(inputY)
dotXY = Dot(axes=2)([embX, embY])
dotXY = Reshape((1,))(dotXY)
outputY = Activation('sigmoid')(dotXY)

model = Model([inputX, inputY], outputY)
model.compile(optimizer = "adam", loss="binary_crossentropy")
model.summary()

# 학습
hist = model.fit([X[:, 0], X[:, 1]], X[:, 2],
                 batch_size=BATCH_SIZE,
                 epochs=NUM_EPOCHS)

# # Embedding (left side) layer의 W를 저장해 둔다
# with open('data/embedding_W.pickle', 'wb') as f:
#     pickle.dump(model.layers[2].get_weights(), f, pickle.HIGHEST_PROTOCOL)

KeyError: 'locat'

In [ ]:
# SGNS로 pre-trained된 embedding vector를 이용해서
# IMDB 리뷰 문서들을 분류한다 (감성분석).
# -----------------------------------------------
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pickle
import nltk

# # 전처리가 완료된 IMDB 데이터를 읽어온다.
# with open('dataset/preprocessed_review.pickle', 'rb') as f:
#     reviews, label, word2idx, idx2word = pickle.load(f)

# # pre-trained embedding layer의 W를 읽어온다.
# with open('dataset/embedding_W.pickle', 'rb') as f:
#     We = pickle.load(f)

# 학습 데이터를 vocabulary의 인덱스 (수치)로 표현한다.
X = []
for review in reviews:
    X.append([word2idx[w] for w in nltk.word_tokenize(review)])
             
# 학습, 시험 데이터로 분리한다.
x_train, x_test, y_train, y_test = train_test_split(X, label, test_size=0.2)


vocab_size = len(word2idx) + 1  # 사전의 크기
max_length = 200                # 한 개 리뷰 문서의 최대 단어 길이
   
# 1개 리뷰 문서의 단어 개수를 max_length = 200으로 맞춘다.
# 200개 보다 작으면 padding = 0을 추가하고, 200개 보다 크면 뒷 부분을 자른다.
x_train = sequence.pad_sequences(x_train, maxlen=max_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_length)

# Deep Learning architecture parameters
batch_size = 512
embedding_dims = 64
num_kernels = 260        # convolution filter 개수
kernel_size = 3          # convolution filter size
hidden_dims = 300
epochs = 10

xInput = Input(batch_shape = (None, max_length))
emb = Embedding(vocab_size, embedding_dims)(xInput)
emb = Dropout(0.5)(emb)
conv = Conv1D(num_kernels, kernel_size, padding='valid', activation='relu', strides=1)(emb)
conv = GlobalMaxPooling1D()(conv)
ffn = Dense(hidden_dims)(conv)
ffn = Dropout(0.5)(ffn)
ffn = Activation('relu')(ffn)
ffn = Dense(1)(ffn)
yOutput = Activation('sigmoid')(ffn)

model = Model(xInput, yOutput)
model.compile(loss='binary_crossentropy', optimizer='adam')
print(model.summary())

# SKNS에서 학습한 We를 적용한다. (pre-trained)
model.layers[1].set_weights(We)

# 학습한다. (fine-tune)
hist = model.fit(x_train, y_train, 
                 batch_size=batch_size, 
                 epochs=epochs,
                 validation_data = (x_tewest, y_test))

# Loss history를 그린다
plt.plot(hist.history['loss'], label='Train loss')
plt.plot(hist.history['val_loss'], label = 'Test loss')
plt.legend()
plt.title("Loss history")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

# 성능 확인
y_pred = model.predict(x_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
print ("Test accuracy:", accuracy_score(y_test, y_pred))